In [1]:
import pandas as pd

file_path = '../../aspect_modelling/lda/5star_pyABSA_updated.csv'

# Read the JSON file into a DataFrame
df_pyabsa= pd.read_csv(file_path)

# Display the DataFrame
df_pyabsa.head()


,Unnamed: 0,sentence,tokens,aspect,sentiment,confidence,topics
0,32670,man pool kind thought time went pool want exac...,"['man', 'pool', 'kind', 'thought', 'time', 'we...",['pool'],['Negative'],[0.8258000000000001],"[(0, 'Negative', 0.8258000000000001)]"
1,32671,best hotel rate one best hotel stay around wor...,"['best', 'hotel', 'rate', 'one', 'best', 'hote...","['room', 'staff']","['Negative', 'Negative']","[0.998, 0.9956]","[(2, 'Negative', 0.998), (4, 'Negative', 0.9956)]"
2,32672,team hotel perfect nice yet comfort stay ideal...,"['team', 'hotel', 'perfect', 'nice', 'yet', 'c...",['comfort'],['Positive'],[0.9785],"[(4, 'Positive', 0.9785)]"
3,32673,perfect husband frequent travel stay bay sand ...,"['perfect', 'husband', 'frequent', 'travel', '...","['look', 'comfort place', 'staff']","['Positive', 'Negative', 'Negative']","[0.9711000000000001, 0.5018, 0.5366000000000001]","[(4, 'Positive', 0.9711000000000001), (4, 'Neg..."
4,32674,warm hotel trip got flu hotel staff met strait...,"['warm', 'hotel', 'trip', 'got', 'flu', 'hotel...",['staff'],['Negative'],[0.9908],"[(4, 'Negative', 0.9908)]"


In [2]:
import pandas as pd
import ast
import re

# Assuming your data is in a CSV file named 'your_file.csv'
df = pd.read_csv('../../../data/textblob/dp_textblob_5_star.csv')

# Function to get the polarity based on the threshold
def get_polarity(topic_number, threshold=0):
    if topic_number == threshold:
        return "Neutral"
    elif topic_number > threshold:
        return "Positive"
    else:
        return "Negative"

# Create a new DataFrame to store aspect, topic number, and sentiment
new_df = pd.DataFrame(columns=['topic_number', 'aspect', 'sentiment'])

# Iterate through each row and update the new_df
for index, row in df.iterrows():
    aspect_sentiment_str = row['aspect_sentiment']
    
    # Add quotes around words in the string representation
    aspect_sentiment_str_fixed = re.sub(r'(\b\w+\b)', r'"\1"', aspect_sentiment_str)
    
    try:
        aspect_sentiment_list = ast.literal_eval(aspect_sentiment_str_fixed)
        for aspect_entry in aspect_sentiment_list:
            aspect = aspect_entry[0]
            topic_number = aspect_entry[2]
            polarity = df.at[index, f'topic_{topic_number}']
            sentiment = get_polarity(polarity)
            
            # Append to new_df
            new_df.loc[len(new_df)] = {'topic_number': topic_number, 'aspect': aspect, 'sentiment': sentiment}
            
            # print(f"Aspect: {aspect}, Topic Number: {topic_number}, Sentiment: {sentiment}")
    except Exception as e:
        print(f"Error processing row {index}: {e}")

# Group by 'topic_number' and 'sentiment', then count occurrences
textblob_sentiment_counts = new_df.groupby(['topic_number', 'sentiment']).size().unstack(fill_value=0)

# Display the counts
print("Sentiment Counts:")
textblob_sentiment_counts


Sentiment Counts:


sentiment,Negative,Neutral,Positive
topic_number,,,
0,2806,2520,44434
1,557,908,6582
2,1680,2859,21458
3,2840,2882,22575
4,508,1470,4802


In [3]:
import pandas as pd
import ast

# Assuming your data is in a CSV file named 'your_other_file.csv'
other_df = pd.read_csv('../../aspect_modelling/lda/5star_pyABSA_updated.csv')

# Create a new DataFrame to store aspect, topic number, and sentiment
other_new_df = pd.DataFrame(columns=['topic_number', 'sentiment'])

# Iterate through each row and update the other_new_df
for index, row in other_df.iterrows():
    topics_str = row['topics']
    
    try:
        topics_list = ast.literal_eval(topics_str)
        for topic_entry in topics_list:
            topic_number = topic_entry[0]
            sentiment = topic_entry[1]
            
            # Append to other_new_df
            other_new_df.loc[len(other_new_df)] = {'topic_number': topic_number, 'sentiment': sentiment}
            
            #print(f"Topic Number: {topic_number}, Sentiment: {sentiment}")
    except Exception as e:
        print(f"Error processing row {index}: {e}")


# Group by 'topic_number' and 'sentiment', then count occurrences
pyabsa_sentiment_counts = other_new_df.groupby(['topic_number', 'sentiment']).size().unstack(fill_value=0)

# Display the counts
print("Sentiment Counts:")
pyabsa_sentiment_counts

Sentiment Counts:


sentiment,Negative,Neutral,Positive
topic_number,,,
0,3092,4775,13798
1,1471,3361,7427
2,3346,1219,6354
3,392,313,1051
4,6350,4867,23241


In [4]:
pyabsa_sentiment_counts

# Calculate the total number of sentiments for each topic
pyabsa_sentiment_counts['Total'] = pyabsa_sentiment_counts.sum(axis=1)

# Calculate the percentage of positive, neutral, and negative sentiments for each topic
df_percentage = pyabsa_sentiment_counts[['Negative', 'Neutral', 'Positive']].div(pyabsa_sentiment_counts['Total'], axis=0) * 100

# Calculate the total number of sentiments for each topic
pyabsa_sentiment_counts['Total'] = pyabsa_sentiment_counts.sum(axis=1)

# Calculate the percentage representation of each topic with respect to the 'Total' column
pyabsa_sentiment_counts['Percentage'] = pyabsa_sentiment_counts['Total'] / pyabsa_sentiment_counts['Total'].sum() * 100

# Print the result
print(pyabsa_sentiment_counts["Percentage"])

# Print the result
print(df_percentage)

topic_number
0    26.728105
1    15.123925
2    13.470767
3     2.166377
4    42.510826
Name: Percentage, dtype: float64
sentiment      Negative    Neutral   Positive
topic_number                                 
0             14.271867  22.040157  63.687976
1             11.999347  27.416592  60.584061
2             30.643832  11.164026  58.192142
3             22.323462  17.824601  59.851936
4             18.428231  14.124441  67.447327


In [5]:
textblob_sentiment_counts

# Calculate the total number of sentiments for each topic
textblob_sentiment_counts['Total'] = textblob_sentiment_counts.sum(axis=1)

# Calculate the percentage of positive, neutral, and negative sentiments for each topic
df_percentage = textblob_sentiment_counts[['Negative', 'Neutral', 'Positive']].div(textblob_sentiment_counts['Total'], axis=0) * 100

# Calculate the total number of sentiments for each topic
textblob_sentiment_counts['Total'] = textblob_sentiment_counts.sum(axis=1)

# Calculate the percentage representation of each topic with respect to the 'Total' column
textblob_sentiment_counts['Percentage'] = textblob_sentiment_counts['Total'] / textblob_sentiment_counts['Total'].sum() * 100

# Print the result
print(textblob_sentiment_counts["Percentage"])

# Print the result
print(df_percentage)


topic_number
0    41.856983
1     6.768954
2    21.868087
3    23.802794
4     5.703182
Name: Percentage, dtype: float64
sentiment      Negative    Neutral   Positive
topic_number                                 
0              5.639068   5.064309  89.296624
1              6.921834  11.283708  81.794458
2              6.462284  10.997423  82.540293
3             10.036400  10.184825  79.778775
4              7.492625  21.681416  70.825959
